# Homework 3

Please read the instructions before starting.

- Only use array manipulation functions from ```numpy```. (Similar to last homework)
- You can use ```PIL``` for reading images and ```ipywidgets``` and ```display``` to display them.
- Use ```numpy``` operations and arrays as much as possible for performance criteria. Try to avoid using for-loops as they will drastically slow down your implementations for large-scale images. Slow implementations will have a penalty during grading.
- You can overwrite the template as long as the above conditions are not violated and the functionality is kept the same. Keep in mind that you will **only** submit the ```hw3.ipynb``` notebook and ```last_homework``` module (there is no report).

 Fill the the marked areas in the cells for each question. 

- - -

## Question 1 [40pt]

In this question, you will implement Tomasi Kanade corner detection algorithm. First implement the ```calculate_gradients``` functions to obtain x and y directional derivatives of the input image. Make sure that you pad the input image so that the derivative arrays have the same shape as the input array.

> **Note**: You need to fill ```last_homework``` Python module first. You can put the implementations you have made in the last homework.

Then, implement ```calculate_gradient_covariance_matrix``` function to obtain gradient covariance matrix, implement ```calculate_eigen_values_and_vectors``` function, and finally, implement ```tomasi_kanade_corners```.


In [8]:
from typing import Tuple
from PIL import Image
import numpy as np

from visualizers.utils import image_to_array
from visualizers.corners import CornerRenderer
from last_homework import sobel_horizontal, sobel_vertical, gaussian_filter, apply_filter


def calculate_gradients(image_array: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """ Use Sobel operators to calculate image derivatives. Use padding
    such that the filtered images have the same shape as the input.

    Args:
        image_array (np.ndarray): Grayscale image array of shape (H, W)

    Returns:
        Tuple[np.ndarray, np.ndarray]: x and y derivatives of shape (H, W)
    """
    x_derivative = sobel_horizontal(image_array)
    y_derivative = sobel_vertical(image_array)
    return x_derivative, y_derivative
    raise NotImplementedError



def calculate_gradient_covariance_matrix(
        x_derivative: np.ndarray,
        y_derivative: np.ndarray,
        kernel_height: int,
        kernel_width: int) -> np.ndarray:
    """ Return gradient covariance matrix from derivative arrays using 
    Gaussian averaging and same padding (the output of the filter has the 
    same shape).

    Args:
        x_derivative (np.ndarray): x derivative of shape (H, W)
        y_derivative (np.ndarray): y derivative of shape (H, W)
        kernel_height (int): Height of the averaging kernel
        kernel_width (int): Width of the averaging kernel

    Returns:
        np.ndarray: Gradient Covariance Matrix of shape (H, W, 2, 2)
    """
    x_derivative = gaussian_filter(x_derivative, (kernel_height, kernel_width), 1)
    y_derivative = gaussian_filter(y_derivative, (kernel_height, kernel_width), 1)
    covariance_matrix = np.zeros((x_derivative.shape[0], x_derivative.shape[1], 2, 2))
    for i in range(x_derivative.shape[0]):
        for j in range(x_derivative.shape[1]):
            covariance_matrix[i][j] = np.array([[x_derivative[i][j] ** 2, x_derivative[i][j] * y_derivative[i][j]], [x_derivative[i][j] * y_derivative[i][j], y_derivative[i][j] ** 2]])
    return covariance_matrix
    raise NotImplementedError


def calculate_eigen_values_and_vectors(covariance_matrix: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """ Calculate the eigen values and vectors for every pixel.
        **Remember**: Covariance matrix is symmetric
        **Note**: You can use numpy.linalg for eigen value/vector calculations

    Args:
        covariance_matrix (np.ndarray): Gradient covariance matrix of shape (H, W, 2, 2)

    Returns:
        Tuple[np.ndarray, np.ndarray]: Eigen values of shape (H, W, 2) and eigen vectors of 
            shape (H, W, 2, 2)
    """

    eigen_values = np.zeros((covariance_matrix.shape[0], covariance_matrix.shape[1], 2))
    eigen_vectors = np.zeros((covariance_matrix.shape[0], covariance_matrix.shape[1], 2, 2))
    for i in range(covariance_matrix.shape[0]):
        for j in range(covariance_matrix.shape[1]):
            eigen_value, eigen_vector = np.linalg.eig(covariance_matrix[i, j])
            eigen_values[i, j] = eigen_value
            eigen_vectors[i, j] = eigen_vector
    return eigen_values, eigen_vectors
    raise NotImplementedError


def tomasi_kanade_corners(eigen_values: np.ndarray) -> np.ndarray:
    """ Return corner point by applying the Tomasi Kanade algorithm.

    Args:
        eigen_values (np.ndarray): Eigen values of the gradient covariance 
            matrix (H, W, 2)

    Returns:
        np.ndarray: Corner indices of shape (C, 2). For every corner c in C,
            the return array contains (y, x) indices of that corner point.
    """
    R = np.zeros((eigen_values.shape[0], eigen_values.shape[1]))
    for i in range(eigen_values.shape[0]):
        for j in range(eigen_values.shape[1]):
            R[i, j] = eigen_values[i, j, 0] * eigen_values[i, j, 1] - 0.04 * (eigen_values[i, j, 0] + eigen_values[i, j, 1]) ** 2
    threshold = np.max(R) * 0.01
    corner_indices = np.zeros((0, 2))
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            if R[i, j] > threshold:
                corner_indices = np.vstack((corner_indices, np.array([i, j])))
    return corner_indices
    raise NotImplementedError

> Make sure that your functions work as expected before continuing the rendering phase

You can interactively observe the window of the hovered point on the image and its derivative plot.
You can change the kernel size of the gaussian averaging if you want.

In [9]:
image = Image.open("anıtkabir.jpeg")
image_array = image_to_array(image)
gaussian_kernel_size = (15, 15) # Parameter
kernel_height, kernel_width = gaussian_kernel_size

# Image Derivatives
x_derivative, y_derivative = calculate_gradients(image_array)
# Covariance Matrix
cov_matrix = calculate_gradient_covariance_matrix(
    x_derivative,
    y_derivative,
    kernel_height,
    kernel_width)
# Eigen Values and Vectors
eigen_vals, eigen_vecs = calculate_eigen_values_and_vectors(cov_matrix)
# Corner Points
corner_points = tomasi_kanade_corners(eigen_vals)

# Interactive Visualization
CornerRenderer(
    "anıtkabir.jpeg",
    [kernel_height, kernel_width],
    [250, 250],
    x_derivative=x_derivative,
    y_derivative=y_derivative,
    eigen_values=eigen_vals,
    eigen_vectors=eigen_vecs,
    corner_indices=corner_points
)()


TypeError: tuple indices must be integers or slices, not tuple

## Question 2 [40pt]

You will be implementing line detection using Hough Transform. First, we need to obtain a binary image of edges. Implement ```binary_edge_image``` function using the Sobel operator that you have implemented in the second homework. You can place your previous implementation into ```last_homework.py``` Python module and import from there. Note: ```last_homework.py``` module will not be evaluated.

Second, you will implement image (binary edge image) to Hough transformation. This closure returns a function that expects array of x and y coordinates from the image and returns the hough array. Implement ```image_to_hough``` function.

Finally, you will implement ```hough_to_image``` closure that returns a function that takes a $\rho$ and $\theta$ values as input and returns a 4 tuple (x1, y1, x2, y2) of a the endpoints of a corresponding line segment. This function allows us to draw lines on the image.

The outputs of the closures will be used as callbacks at the interactive visualization object.

**Note**: The reason why we use Python closure is to perform initial computations once instead of at every call of the callback.

In [ ]:
from typing import Callable, Tuple, List
from PIL import Image
import numpy as np
from dataclasses import dataclass, asdict

from visualizers.hough import HoughRenderer
from visualizers.utils import image_to_array
from last_homework import sobel_horizontal, sobel_vertical


# You can change the configuration but make sure that your
# implementation works with the the default values
@dataclass
class HoughConfig:
    width: int = 300
    height: int = 300
    n_rho: int = 150
    n_theta: int = 150


def binary_edge_image(image_array: np.ndarray) -> np.ndarray:
    """ Convert 2D gray image array into 2D binary edge image array
        **Note**: You can use Sobel operators.

    Args:
        image_array (np.ndarray): 2D image array

    Returns:
        np.ndarray: 2D binary (either np.bool (true, false) or np.int (0, 1)) array. 
            Image shape is preserved.
    """
    #  /$$$$$$$$ /$$$$$$ /$$       /$$
    # | $$_____/|_  $$_/| $$      | $$
    # | $$        | $$  | $$      | $$
    # | $$$$$     | $$  | $$      | $$
    # | $$__/     | $$  | $$      | $$
    # | $$        | $$  | $$      | $$
    # | $$       /$$$$$$| $$$$$$$$| $$$$$$$$
    # |__/      |______/|________/|________/
    raise NotImplementedError


def image_to_hough(width: int, height: int, n_rho: int, n_theta: int
                   ) -> Callable[[np.ndarray, np.ndarray], np.ndarray]:
    """ Closure that returns a function that transform array of points 
    to Hough space.

    Args:
        width (int): Image width
        height (int): Image height
        n_rho (int): Number of rhos in the Hough space
        n_theta (int): Number of thetas in the Hough space

    Returns:
        Callable[[np.ndarray, np.ndarray], np.ndarray]: Transformer function
    """
    #  /$$$$$$$$ /$$$$$$ /$$       /$$
    # | $$_____/|_  $$_/| $$      | $$
    # | $$        | $$  | $$      | $$
    # | $$$$$     | $$  | $$      | $$
    # | $$__/     | $$  | $$      | $$
    # | $$        | $$  | $$      | $$
    # | $$       /$$$$$$| $$$$$$$$| $$$$$$$$
    # |__/      |______/|________/|________/

    def transform(x_cords: np.ndarray, y_cords: np.ndarray) -> np.ndarray:
        """ Transform array of points from edge image to Hough space
            **Note**: Try to avoid using loops. Instead use numpy operations and make use
            of auto-broadcasting. Slow implementations will be penalized.

        Args:
            x_cords (np.ndarray): X coordinates of the points in 1D integer array
            y_cords (np.ndarray): Y coordinates of the points in 1D integer array

        Returns:
            np.ndarray: 2D Hough array
                - Range of rho: [-d/2, d/2] where "d" denotes diagonal length 
                of the image
                - Range of theta: [-π, π]
        """
        #  /$$$$$$$$ /$$$$$$ /$$       /$$
        # | $$_____/|_  $$_/| $$      | $$
        # | $$        | $$  | $$      | $$
        # | $$$$$     | $$  | $$      | $$
        # | $$__/     | $$  | $$      | $$
        # | $$        | $$  | $$      | $$
        # | $$       /$$$$$$| $$$$$$$$| $$$$$$$$
        # |__/      |______/|________/|________/
        raise NotImplementedError

    return transform


def hough_to_image(width: int, height: int) -> Callable[[float, float], Tuple[int, int, int, int]]:
    """ Closure that returns a function for calculating the end points of a line
    for the single point from the Hough space 

    Args:
        width (int): Image width
        height (int): Image height

    Returns:
        Callable[[float, float], Tuple[int, int, int, int]]: Transformer function
    """
    #  /$$$$$$$$ /$$$$$$ /$$       /$$
    # | $$_____/|_  $$_/| $$      | $$
    # | $$        | $$  | $$      | $$
    # | $$$$$     | $$  | $$      | $$
    # | $$__/     | $$  | $$      | $$
    # | $$        | $$  | $$      | $$
    # | $$       /$$$$$$| $$$$$$$$| $$$$$$$$
    # |__/      |______/|________/|________/

    def transform(rho: float, theta: float) -> Tuple[int, int, int, int]:
        """ Calculate the end points of a line on the image

        Args:
            rho (float): rho value on the Hough space
            theta (float): theta value on the Hough space

        Returns:
            Tuple[int, int, int, int]: End points of a line.
                (x1, y1, x2, y2).
                - range of x: [0, image width]
                - range of y: [0, image height]
        """
        #  /$$$$$$$$ /$$$$$$ /$$       /$$
        # | $$_____/|_  $$_/| $$      | $$
        # | $$        | $$  | $$      | $$
        # | $$$$$     | $$  | $$      | $$
        # | $$__/     | $$  | $$      | $$
        # | $$        | $$  | $$      | $$
        # | $$       /$$$$$$| $$$$$$$$| $$$$$$$$
        # |__/      |______/|________/|________/
        raise NotImplementedError

    return transform


### Run the cell below to interact between the binary edge and Hough spaces.

> Note: You must implement the above functions before running the visualization

- Hover your mouse cursor over the edge image and hough space

In [ ]:
pil_image = Image.open("anıtkabir.jpeg").resize((HoughConfig.width, HoughConfig.height))
image_array = image_to_array(pil_image)

interactive_renderer = HoughRenderer(
    image_array,
    binary_edge_image(image_array),
    HoughConfig.n_rho,
    HoughConfig.n_theta,
)

# Add callbacks
interactive_renderer.add_edge_to_hough_callback(image_to_hough(**asdict(HoughConfig())))
interactive_renderer.add_hough_to_edge_callback(hough_to_image(width=HoughConfig.width, height=HoughConfig.height))

# run interactive visualization
interactive_renderer()

Now, you will implement the full algorithm that draws multiple lines on the image. You should use the above implementations to fill the functions (```make_hough_array```, ```make_line_from_max_hough_points```) below.

In [ ]:
pil_image = Image.open("anıtkabir.jpeg").resize((HoughConfig.width, HoughConfig.height))
image_array = image_to_array(pil_image)
edge_image = binary_edge_image(image_array)


def make_hough_array(binary_edge_image: np.ndarray) -> np.ndarray:
    """ Form a Hough array using the edge points in the <binary_edge_image>
        **Note**: You should use the closure "image_to_hough" in this implementation
    Args:
        binary_edge_image (np.ndarray): 2D binary array of edges of shape (H, W)

    Returns:
        np.ndarray: Hough array of shape (H, W)
    """
    #  /$$$$$$$$ /$$$$$$ /$$       /$$
    # | $$_____/|_  $$_/| $$      | $$
    # | $$        | $$  | $$      | $$
    # | $$$$$     | $$  | $$      | $$
    # | $$__/     | $$  | $$      | $$
    # | $$        | $$  | $$      | $$
    # | $$       /$$$$$$| $$$$$$$$| $$$$$$$$
    # |__/      |______/|________/|________/
    raise NotImplementedError


def make_line_from_max_hough_points(hough_array: np.ndarray, n_points: int
                                    ) -> List[Tuple[int, int, int, int]]:
    """ Select the highest <n_points> points on the Hough space and
    return a list of line segment endpoints of them.
    **Note**: You should use the closure "hough_to_image" in this implementation

    Args:
        hough_array (np.ndarray): Hough array of shape (H, W) 
        n_points (int): Number of points to select from hough space

    Returns:
        List[Tuple[int, int, int, int]]: List of endpoints of the line segments
            corresponding to selected points from the Hough space
    """
    #  /$$$$$$$$ /$$$$$$ /$$       /$$
    # | $$_____/|_  $$_/| $$      | $$
    # | $$        | $$  | $$      | $$
    # | $$$$$     | $$  | $$      | $$
    # | $$__/     | $$  | $$      | $$
    # | $$        | $$  | $$      | $$
    # | $$       /$$$$$$| $$$$$$$$| $$$$$$$$
    # |__/      |______/|________/|________/
    raise NotImplementedError


> Make sure that your functions work as expected before continuing the rendering phase

- Run the cell below to visualize the line detection algorithm that you have implemented.

- Run the cell below that to create a slider that allows you to choose number of maximum points from Hough space interactively.

In [ ]:
full_renderer = HoughRenderer(
    image_array,
    edge_image,
    HoughConfig.n_rho,
    HoughConfig.n_theta,
)

hough_array = make_hough_array(edge_image)

# Draw hough space
full_renderer.draw_hough_space(hough_array)
# Draw 20 lines
full_renderer.batch_draw_lines(
    make_line_from_max_hough_points(hough_array, 20)
)

full_renderer()


In [ ]:
# Make a slider and add a callback to draw lines on edge image
full_renderer.n_points_selector(
    lambda n_points: full_renderer.batch_draw_lines(
        make_line_from_max_hough_points(hough_array, n_points["new"]))
        )

## Question 3 [20pt]

Fill the markdown cells below with your answers



### In the corner detection algorithm
- Why do we use minimum of the eigen values and not the maximum?



- What is the difference between using a Gaussian weighting and uniform weighting during the calculation of the gradient covariance matrix?



- Briefly propose (do not implement) a procedure for automatically finding threshold for eigen values



### Hough Transform
- Briefly propose (do not implement) a parametrization that can draw line segments (line width finite length) instead of just lines (infinite length). What is the dimensionality of the Hough space under the proposed parameterization?

